In [ ]:
def simplex_model():
    X= tokenizer(p)   #posts
    for id in author_ids:       #each author has only one Z,theta and b
        hidden_state_1_output=bert(X)   #using pretrained-Bert-model as regressor
        Z= multiply(Z,topic.T)+multiply(hidden_state_1_output,topic.T)*(1-beta) +multiply(Z,topic.T)*beta  # moving average  (1-beta)*Zt+beta*Zt-1
    
        hidden_state_2_output=linear(theta)
        hidden_state_2_output=clamp(hidden_state_2_output,-0.5,0.5)
        theta=hidden_state_2_output*(1-beta)+theta*beta
    
        hidden_state_3_output=linear(b)
        hidden_state_3_output=clamp(hidden_state_3_output,0,1)
        b= hidden_state_3_output*(1-beta)+b*beta


        Y=Z*[0.5+theta,0.5-theta].T+b

    return Y, Z, theta, b


def train():
    for epoch in n_epochs:
        for (post,id,t) in dataloader:
            Y_hat, Z, theta, b =simplex_model(post,id,t)

        loss=(Y-Y_hat)**2 + lambd1 * l2_penalty(theta) + lambd2 * l2_penalty(b)

        optimizer.zero_grad()
        loss.backward
        optimizer.step()

In [ ]:
def complex_model():
    X= tokenizer(p)  
    for id in author_ids:       
        hidden_state_1_output=bert(X)   
        Z= multiply(Z,topic.T)+multiply(hidden_state_1_output,topic.T)*(1-beta) +multiply(Z,topic.T)*beta 
    
        hidden_state_2_output=linear(theta)
        hidden_state_2_output=clamp(hidden_state_2_output,-0.5,0.5)
        theta=hidden_state_2_output*(1-beta)+theta*beta
    
        hidden_state_3_output=linear(b)
        hidden_state_3_output=clamp(hidden_state_3_output,0,1)
        b= hidden_state_3_output*(1-beta)+b*beta

        Y=Y=Z*[0.5+theta,0.5-theta].T+b

        return Y, Z, theta, b

def train():
    for epoch in n_epochs:
        for (post,id,t) in dataloader:
            Y_hat, Z, theta, b =complex_model(post,id,t)

        for y in Y:
            miu_z_given_y = sum(Z)/n
            sigma_z_given_y = (sum((Z-miu_y)**2)/n-1)

        for id in author_ids:
            for y in Y_hat:
                sigma_z = tensor.append(Z.std()).mean()
 
        sigma_y_hat=y_hat.var()
        simga_z_given_y_hat = get_variance(sigma_y_hat,sigma_z,theta)
        miu_z_given_y_hat = get_mean(Z,y,theta,b)

        loss=nn.KLDivLoss(torch.normal(miu_z_given_y,sigma_z_given_y),torch.normal(miu_z_given_y_hat,simga_z_given_y_hat))

        optimizer.zero_grad()
        loss.backward
        optimizer.step()

In [ ]:
def get_variance():
    var_inverse=tensor[[1/sigma_y_hat*(0.5+theta)**2+1/sigma_z , (0.25-theta**2)*1/sigma_y_hat]
                       [(0.25-theta**2)*1/sigma_y_hat , 1/sigma_y_hat*(0.5-theta)**2+1/sigma_z]]
    variance= torch.inverse(var_inverse)
    return variance

def get_mean():
    miu=mean((y_hat-b)*[0.5+theta,0.5-theta].T+Z)
    return miu